## Notes On Scale


### The Upshot

The original Project Prototype (https://github.com/sergeypine/WeatherLearner/tree/capstone-prototype/webapp) operates on the data scale that is comparable to what is expected in the final deployment. Therefore, there is no need for further scaling up of the solution.

### Some Numbers

At the heart of the solution are 16 models: for 4 quantities (`Temperature`, `WindSpeed`, `Precipitation` and `Clearness`) and for 4 lookahead intervals (`6h`, `12h`, `18h` and `24h`). 

Each of those models were pre-trained on 10 years of Hourly Weather Data from the Target Location (Chicago) as well as a model-specific subset of the 11 neighboring geographic locations (around the US MidWest). With that in mind, let us estimate the total amount of data involved.

- A single location dataset countains 10 years * 365 days * 24 hours = **~87K records**
- Each hourly record has **24** numeric variables(features) giving us up to **~2M** variables per location
- _Therefore, for all 12 locations, we end up with **~1M records** and **~24M** variables_
- Allowing the standard 4 bytes per variable, we end up with _under **100Mb**_ of total data

NOTE that the estimates above are upper bounds because each of the 16 models uses only a subset of features and a subset of locations. Furthermore, we do not train on all data but only on the 70% of the data that is designated as the Training Set (the remaining 30% are left aside for testing and validation)

To produce the prototype, a single _Dell XPS 13 7390_ Laptop with **16Gb / 12 cores** was used. That is similar to AWS EC2 _t3.2xlarge_ or _c4.2xlarge_ instance types.

Using that hardware, training of all models takes **~1h** while loading and using all those models for latest forecast generation takes **under 1 minute**. These are acceptable numbers because training of the models is done very rarely (once per deployment) and predictions are made asynchronously (every hour) and then cached.

At the time of writing of this document, there is the possibility of doubling the amount of data on which models are trained from 10 years to 20 years. That would still leave us with _under **200Mb**_ of data. By the current standards that is simply not much data.

### Possible Scalability Improvements

While, as per above, no scalability-targeting improvements are necessary, let it be stated that they are nevertheless possible in theory. 

Specifically, since there are multiple (16) models underlying the solution, these models can be subdivided into sets to be handled by separate Nodes/Containers/Workers as follows:

- For the (very rare) training of the models, separate Nodes can be used to train different models. The trained models can then be uploaded to shared storage (such as AWS S3). The training data also can reside in that shared storage. This approach can cut the total training time ~16 X

- For the prediction generation stage, we can also have multiple Nodes with each handling only a subset of models. For example, Node1 would predict Precipitation in 6h, Node2 would predict Precipitation in 12h, Node3 would predict Temperature in 6h etc. Each of these predictions can be uploaded to shared storage for aggregation and serving by the Nodes running the Web Application Server.

- Finally, we can also parallelize the downloading of recent Weather data (which takes a couple of minutes for the 12 locations used in the Prototype). For example, Node 1 could download data for all locations East of Chicago, Node 2 for all location West etc.

All of the above improvements would constitute Horizontal Scaling. 